In [1]:
from fastapi import FastAPI, HTTPException, WebSocket, WebSocketDisconnect
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from langchain_ollama import OllamaLLM
import os
from langchain_postgres.vectorstores import PGVector
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings


In [2]:
# Modelos de Pydantic
class Item(BaseModel):
    input: str

In [3]:
load_dotenv()

PGV_USER=os.getenv("PGV_USER")
PGV_PASSWORD=os.getenv("PGV_PASSWORD")
PGV_HOST=os.getenv("PGV_HOST")
PGV_PORT=os.getenv("PGV_PORT")
PGV_DATABASE_NAME=os.getenv("PGV_DATABASE_NAME")

EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL")
UCN_MODEL_NAME=os.getenv('UCN_MODEL_NAME')


In [4]:
app = FastAPI()
# CORS middleware. Restringir acceso antes de producción
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [7]:
model = OllamaLLM(model=UCN_MODEL_NAME,host="localhost",port=11434)

custom_prompt_template = """
Contexto: {context}
Pregunta: {question}

Responde siempre en español a no ser que te pidan lo contrario.
Respuesta útil:
"""
moder_for_embedding = FastEmbedEmbeddings(model_name=EMBEDDING_MODEL)

prompt = PromptTemplate(template=custom_prompt_template, input_variables=['context', 'question'])


Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]


In [8]:
def initialize_qa():
    
    connection = f'postgresql+psycopg://{PGV_USER}:{PGV_PASSWORD}@{PGV_HOST}:{PGV_PORT}/{PGV_DATABASE_NAME}'
    
    vector_store = PGVector(
                embeddings=moder_for_embedding,
                connection=connection,
                use_jsonb=True,
            )
    retriever = vector_store.as_retriever(kwargs={"k":2})

    qa = RetrievalQA.from_chain_type(
        llm=model,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt, 
                           "verbose": True
                           }
    )
    
    return qa

In [9]:
qa = initialize_qa()

Exception: Failed to create vector extension: (psycopg.OperationalError) [Errno 11001] getaddrinfo failed
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
connection = f'postgresql+psycopg://{PGV_USER}:{PGV_PASSWORD}@{PGV_HOST}:{PGV_PORT}/{PGV_DATABASE_NAME}'

vector_store = PGVector(
            embeddings=moder_for_embedding,
            connection=connection,
            use_jsonb=True,
        )
retriever = vector_store.as_retriever(kwargs={"k":2})

In [10]:
retriever.invoke(input="¿Ques un alumno regular?")

[Document(id='43d7c5a2-46b6-4742-8693-2017bde3d586', metadata={'page': 4, 'source': 'C:\\Users\\claud\\Desktop\\Cursos\\LearningLangChain\\ChatBotUCN\\mapping\\media\\PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf'}, page_content='Art. 6 Estudiante regular: Es quien ingresa a través de los procedimientos de admisión, que establece el Título II| del presente Reglamento, y adscrito a un programa o carrera determinada, cursa estudios conducentes a un grado académico y/o título profesional. La calidad de estudiante regular de la Universidad Católica del Norte se adquiere una vez que la matrícula ha sido formalizada en conformidad a las normas correspondientes. Art. 7. Estudiante egresado/a: Corresponde a estudiante regular que ha aprobado las actividades curriculares establecidas en el libro de carrera o programa respectivo y a quienes solo reste cumplir con las actividades finales exigidas para obtener el grado o titulo respectivo. Art. 8. Estudiante de intercambio saliente: E

In [19]:
@app.get("/")
def read_root():
    return {"message": "¡Hola, mundo!"}

In [20]:
@app.post("/generate")
def process(item: Item):
    input_text = item.input
    
    try:
        response = qa.invoke({"query": input_text})
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
    
    return {"response": response}

In [ ]:
model.stream()

In [ ]:
qa.stream()

In [21]:
@app.websocket("/ws/generate")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    try:
        while True:
            data = await websocket.receive_text()
            try:
                # Usar el método stream() para obtener resultados a medida que se generan
                async for response_chunk in qa.stream({"query": data}):
                    print(response_chunk)  # Esto es útil para depuración
                    result_text = response_chunk['result']
                    
                    # Enviar la respuesta parcial al cliente
                    await websocket.send_json({"response": result_text})
            except Exception as e:
                await websocket.send_json({"error": str(e)})
    except WebSocketDisconnect:
        print("Client disconnected")

In [22]:
if __name__ == '__main__':
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)

c:\Users\claud\Desktop\Cursos\LearningLangChain\thisEnvironment\Lib\site-packages\pygments\regexopt.py:78: RuntimeWarning: coroutine 'Server.serve' was never awaited
  for group in groupby(strings, lambda s: s[0] == first[0])) \


RuntimeError: asyncio.run() cannot be called from a running event loop